In [50]:
import pandas as pd
import numpy as np
import datetime
import os


def assign_lap(df):
    df['LAP'] = 1
    cols = ['NO', 'GAP', 'TIME', 'LAP']
    drivers = df[0].unique()
    data = df.values
    for driver in drivers:
        data[data[:,0] == driver, 3] = data[data[:,0] == driver, 3].cumsum()
    return pd.DataFrame(data=data, columns=cols)

def get_strategies(df):
    arr = df.values
    all_strategies = []
    for row in arr:
        strategy = [row[0]]
        for item in row[1:]:
            try:
                tire, laps = item.split()
                laps = int(laps.replace('(', '').replace(')', ''))
                stint = []
                for i in xrange(laps):
                    stint.append(tire)
                strategy.extend(stint)
            except:
                pass
        all_strategies.append(strategy)
    df = pd.DataFrame(data=all_strategies)
    column_names = ['NAME']
    column_names.extend([i for i in xrange(1, df.shape[1])])
    df.columns = column_names
    df[['NO', 'LAP']] = df[['NO', 'LAP']].astype(int)
    merged = pd.merge(driver_list, df, on='NAME')
    return merged.drop(['NAME', 'DRIVER'], axis=1)



In [20]:
# Load driver list for particular season
driver_list = pd.read_csv('data/2016_drivers.csv')

In [111]:
# Load lap times for all drivers
df = pd.read_csv('data/2016_2_bahrain/tabula-2016_2_lap_history.csv', header=None)
df = assign_lap(df)
df.sort_values(by=['NO', 'LAP'], inplace=True)
df[df['NO'] == 6]

,NO,GAP,TIME,LAP
0,6,0,1:39.458,1
20,6,0,1:38.149,2
40,6,0,1:37.731,3
60,6,0,1:37.483,4
80,6,0,1:37.663,5
100,6,0,1:37.754,6
120,6,0,1:37.787,7
139,6,0,1:37.808,8
158,6,0,1:38.174,9
177,6,0,1:38.297,10


In [112]:
df.NO.unique()
# df.info()

array([3, 6, 7, 8, 9, 11, 12, 19, 20, 21, 22, 26, 27, 33, 44, 47, 55, 77,
       88, 94], dtype=object)

In [86]:
# Convert lap time to seconds
df['TIME'] = df['TIME'].apply(lambda x: x.strip())
df['TIME'] = df['TIME'].apply(lambda x: datetime.datetime.strptime(x,'%M:%S.%f'))
df['timedelta'] = df.TIME - datetime.datetime.strptime('00:00.0','%M:%S.%f')
df['secs'] = df['timedelta'].apply(lambda x: x / np.timedelta64(1, 's'))
df.head()

,NO,GAP,TIME,LAP,timedelta,secs
8,3,10.686,1900-01-01 00:02:27.788,1,00:02:27.788000,147.788
25,3,7.246,1900-01-01 00:02:37.174,2,00:02:37.174000,157.174
43,3,3.906,1900-01-01 00:02:30.687,3,00:02:30.687000,150.687
61,3,6.469,1900-01-01 00:01:45.978,4,00:01:45.978000,105.978
79,3,8.357,1900-01-01 00:01:45.298,5,00:01:45.298000,105.298


In [ ]:
# # Load Tire strategy data
# tire_data = pd.read_csv('data/2016_1_austrailia/2016_1_tires.csv')
# tire_strat = get_strategies(tire_data)

In [ ]:
# tire_strat.head()

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1012 entries, 14 to 1008
Data columns (total 4 columns):
NO      1012 non-null object
GAP     1012 non-null object
TIME    1011 non-null object
LAP     1012 non-null object
dtypes: object(4)
memory usage: 39.5+ KB
